## Runpod용 코드  
VRAM 부족하면 코드가 안 돌아갈 수 있음  
runpod에선 !pip install 실행

In [ ]:
# !pip install rank_bm25
# !pip install transformers

In [1]:
import os
import re
import torch
import random
import pandas as pd
from tqdm import tqdm
from glob import glob
from itertools import combinations
from transformers import AutoTokenizer
from rank_bm25 import BM25Okapi

In [2]:
base_path = './open'
train_path = os.path.join(base_path, 'train_code')
data_path = os.path.join(base_path, 'dataframe')

# 파일들의 경로를 가져오기 위해 glob 사용
dir_list = glob(os.path.join(train_path, '*'))
dir_list.sort()
dir_list[:5]

['./open\\train_code\\problem001',
 './open\\train_code\\problem002',
 './open\\train_code\\problem003',
 './open\\train_code\\problem004',
 './open\\train_code\\problem005']

### 데이터 전처리 함수

In [ ]:
import re

def data_clean(text):
    # 중복 줄 바꿈 제거
    

    # 주석 제거
    text = re.sub(r'//.*', '', text)  # 한 줄 주석 제거
    text = re.sub(r'/\*.*?\*/', '', text, flags=re.DOTALL)  # 여러 줄 주석 제거
    
    # #include 제거
    text = re.sub(r'#include.*', '', text)

    # 기타 제거
    text = re.sub(r'<vector>', '', text)
    text = re.sub(r'vector<.*?>', '', text)
    text = re.sub(r'typedef.*;', '', text)
    text = re.sub(r'template<.*?>', '', text)
    
    # using namespace std 제거
    text = re.sub(r'using namespace std;', '', text)
    
    # 데이터 타입 형식 제거 (int, long, char 등)
    text = re.sub(r'\b(int|long|char|short|float|double|bool|void|unsigned|signed)\b', '', text)
    
    # const 제거
    text = re.sub(r'\bconst\b', '', text)
    
    # pragma 제거
    text = re.sub(r'#pragma.*', '', text)
    
    text = re.sub(r'\n+', '\n', text)

    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\t', ' ')
    
    # 빈 줄 제거
    text = text.strip()
    
    return text

exam = '''
#include<iostream>
<vector>
using namespace std;
typedef long long li;
#define repa(i,a,n) for(int i=(a);i<(n);i++)
#define rep(i,n) for(int i=0;i<(n);i++)
#define df 0
template<class T> void print(const T& t){ cout << t << "\n"; }
template<class T, class... Ts> void print(const T& t, const Ts&... ts) { cout << t; if (sizeof...(ts)) cout << " "; print(ts...); }

int main(){
  int n; cin >>n;
  vector<int> c(n-1),s(n-1),f(n-1);
  rep(i,n-1){
    cin >>c[i] >>s[i] >>f[i];
  }
  rep(j,n){
    int t=0;
    repa(i,j,n-1){
      //      if(df)print(s[i],f[i]);
      if(t<s[i])t=s[i];
      if(t%f[i])t=(t/f[i]+1)*f[i];
      t+=c[i];
    }
    print(t);
  }
}
'''

cleaned_exam = data_clean(exam)
print(cleaned_exam)


### 데이터 풀어서 전처리 후 저장

In [59]:
code_list = []
p_num_list = []


for p_num, problem in enumerate(tqdm(dir_list), start=1):
    for sol in glob.glob(os.path.join(problem, '*')):
        with open(sol, 'r', encoding='utf-8') as f:
            code = f.read()
            code_list.append(data_clean(code))
            p_num_list.append(p_num)

p_df = pd.DataFrame(data={"code": code_list, "p_num": p_num_list})

# CSV 파일로 저장
p_df.to_csv(os.path.join(data_path, "problem_df_2403262033.csv"), index=False)


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [12:48<00:00,  1.54s/it]


In [41]:
pd.set_option('display.max_colwidth', None)

In [60]:
#불러올때
p_df = pd.read_csv(os.path.join(data_path, "problem_df_2403262033.csv"))

In [61]:
p_df.head()

,code,p_num
0,"#define rep(i, n) for( i = 0; i < ()n; ++i) #define FOR(i, a, b) for( i = a; i < ()b; ++i) using ll = ; Inf = 100100100; EPS = 1e-9; main() { cin.tie(0); ios::sync_with_stdio(false); r, d, x; cin >> r >> d >> x; rep (i, 10) { x = x * r - d; cout << x << endl; } return 0; }",1
1,"using ul = ; using ull = ; #define vec vector #define rep(_i,_n) for(ull _i = 0; _i < _n; _i++) #define rep2(_i,_s,_n) for(ull _i = _s; _i < _n; _i++) #define elif else if #define push(_a) push_back(_a) #define fullSort(_a) sort(_a.begin(), _a.end()) #define inputVecI(_name, _n) vec<> _name(_n);rep(_i, _n)cin >> _name[_i]; #define inputVecUL(_name, _n) vec<ul> _name(_n);rep(_i, _n)cin >> _name[_i]; #define inputVecULL(_name, _n) vec<ull> _name(_n);rep(_i, _n)cin >> _name[_i]; #define inputVecS(_name, _n) vec<string> _name(_n);rep(_i, _n)cin >> _name[_i]; #define printVec(_vec) cout << ""{"";rep(i, _vec.size()-1)cout << _vec[i] << "","";cout << _vec[_vec.size()-1];cout << ""}\n""; #define append(_x) push_back(_x) #define mapLoop(_i, _name) for(auto i = _name.begin(); i != _name.end(); i++) main() { ul r, D, x; cin >> r >> D >> x; rep(i, 10){ x = r * x - D; cout << x << endl; } return 0; }",1
2,"using ll = ; using ull = ; using ld = ; #define MOD 1000000007LL #define rep(i, n) for(ll (i) = 0LL;(i) < (ll)(n);(i)++) #define rep2(i, s, e) for(ll (i) = (ll)(s);(i) < (ll)(e);(i)++) #define repi(i, n) for(ll (i) = 0LL;(i) <= (ll)(n);(i)++) #define repi2(i, s, e) for(ll (i) = (ll)(s);(i) <= (ll)(e);(i)++) #define per(i, n) for(ll (i) = (ll)(n) - 1LL;(i) >= 0LL;(i)--) #define per2(i, s, e) for(ll (i) = (ll)(s) - 1LL;(i) >= (ll)(e);(i)--) #define peri(i, n) for(ll (i) = (ll)(n);(i) >= 0LL;(i)--) #define peri2(i, s, e) for(ll (i) = (ll)(s);(i) >= (ll)(e);(i)--) #define iter(i, it) for(auto &(i): (it)) ostream& operator<<(ostream &s, pair<T, U> m) { cout << ""("" << m.first << "", "" << m.second << "")""; return s; } ostream& operator<<(ostream &s, map<T, U> m) { ll c = 0; cout << ""{ ""; iter(i, m) cout << i << (c++ == m.size() - 1 ? "" "" : "", ""); cout << ""}""; return s; } ostream& operator<<(ostream &s, &v) { cout << ""{ ""; rep(i, v.size()) cout << v[i] << (i == v.size() - 1 ? "" "" : "", ""); cout << ""}""; return s; } ostream& operator<<(ostream &s, list<T> &v) { ll c = 0; cout << ""{ ""; iter(i, v) cout << i << (c++ == v.size() - 1 ? "" "" : "", ""); cout << ""}""; return s; } main() { ll r, D, x; cin >> r >> D >> x; rep(i, 10) { x = r * x - D; cout << x << endl; } return 0; }",1
3,"#define endl '\n'; main() { xi, r, d, x2000, aux; aux = 0; xi = 0; cin >> r >> d >> x2000; while(aux < 10){ xi = r * x2000 - d; x2000 = xi; cout << xi << endl; aux++; } }",1
4,"#define FAST ios_base::sync_with_stdio(0); cin.tie(0); cout.tie(0); #define MAXX 10005 #define PI 3.14159265358979323846264338327950 #define ll ll t , n, c[MAXX]; adj[MAXX]; used[MAXX]; main() { FAST; ll a, b, r; cin >> r >> a >> b; c[0] = b; for(ll i = 1; i <= 10; i++){ c[i] = r*c[i-1] -a; } for(ll i = 1; i <= 10; i++){ cout << c[i] << endl; } }",1


### pari 생성 함수

In [62]:
# from itertools import combinations
# import random
# from transformers import AutoTokenizer
# from rank_bm25 import BM25Okapi

def get_pair(inputs, tokenizer):
  codes = inputs['code'].to_list()
  problems = inputs['p_num'].unique().tolist()
  problems.sort()

  tokenized_corpus = [tokenizer.tokenize(code) for code in codes]
  bm25 = BM25Okapi(tokenized_corpus)

  total_positive_pairs = []
  total_negative_pairs = []

  for problem in tqdm(problems):
    solution_codes = inputs[inputs['p_num'] == problem]['code']
    positive_pairs = list(combinations(solution_codes.to_list(), 2))
    positive_pairs = random.sample(positive_pairs, len(positive_pairs) // 20)

    solution_codes_indices = solution_codes.index.to_list()
    negative_pairs = []

    first_tokenized_code = tokenizer.tokenize(positive_pairs[0][0])
    negative_code_scores = bm25.get_scores(first_tokenized_code)
    negative_code_ranking = negative_code_scores.argsort()[::-1]
    ranking_idx = 0

    for solution_code in solution_codes:
      negative_solutions = []
      while len(negative_solutions) < len(positive_pairs) // len(solution_codes):
        high_score_idx = negative_code_ranking[ranking_idx]

        if high_score_idx not in solution_codes_indices:
          negative_solutions.append(inputs['code'].iloc[high_score_idx])
        ranking_idx += 1

      for negative_solution in negative_solutions:
        negative_pairs.append((solution_code, negative_solution))

    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

  positive_code1 = list(map(lambda x:x[0], total_positive_pairs))
  positive_code2 = list(map(lambda x:x[1], total_positive_pairs))

  negative_code1 = list(map(lambda x:x[0], total_negative_pairs))
  negative_code2 = list(map(lambda x:x[1], total_negative_pairs))

  positive_label = [1] * len(positive_code1)
  negative_label = [0] * len(negative_code1)

  positive_code1.extend(negative_code1)
  positive_code2.extend(negative_code2)
  positive_label.extend(negative_label)

  pair_data = pd.DataFrame(data = {
      'code1' : positive_code1,
      'code2' : positive_code2,
      'similar' : positive_label
  })

  pair_data = pair_data.sample(frac=1).reset_index(drop=True)

  return pair_data



### train test splot

In [64]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    p_df,
    test_size = 0.05,
    random_state = 42,
    stratify = p_df['p_num']
)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_df

,code,p_num
0,"a[100010]; main() { n; scanf(""%d"", &n); gap = 0; for ( i = 0; i < n; i++) { scanf(""%d"", &a[i]); gap = max(gap, a[i]); } ch = -1; cc = 0; for ( i = 0; i < n; i++) { if (a[i] == gap)continue; cc1 = min(a[i], gap - a[i]); if (ch < cc1) { ch = cc1; cc = a[i]; } } printf(""%d %d\n"", gap, cc); }",461
1,"#define mod 1000000007 dy[4]={1,0,-1,0}; dx[4]={0,1,0,-1}; > graph; sort_sec( pair<,> &a, pair<,> &b){ if(a.second<b.second){ return true; } return false; } swap( *a, *b){ temp=*a; *a=*b; *b=temp; } main(){ N,M; cin>>N>>M; A(N); for( i=0;i<N;i++){ cin>>A[i]; } priority_queue<> que; for( i=0;i<N;i++){ que.push(A[i]); } while(M!=0){ M=M-1; v=que.top(); que.pop(); mv=v/2; que.push(mv); } ans=0; for( i=0;i<N;i++){ ans+=que.top(); que.pop(); } cout<<ans<<endl; return 0; }",199
2,"#define fix(n) cout<<fixed<<setprecision(n) #define rep(i,n) for ( i = 0; i < (n); ++i) #define sort(a) sort((a).begin(), (a).end()) #define uniq(a) SORT(a);(a).erase(unique((a).begin(), (a).end()), (a).end()) #define reverse(a) reverse((a).begin(), (a).end()) #define ctos(c) string(1, (c)) #define out(d) cout << (d) #define outl(d) std::cout<<(d)<<""\n"" #define Yes() printf(""Yes\n"") #define No() printf(""No\n"") #define YES() printf(""YES\n"") #define NO() printf(""NO\n"") #define ceil(x, y) ((x + y - 1) / (y)) #define debug(x) cerr << #x << "": "" << (x) << '\n' #define debug2(x, y) cerr << #x << "": "" << (x) << "", "" << #y << "": "" << (y) << '\n' #define debug3(x, y, z) cerr << #x << "": "" << (x) << "", "" << #y << "": "" << (y) << "", "" << #z << "": "" << (z) << '\n' #define dbg(v) for (size_t _ = 0; _ < v.size(); ++_){ cerr << #v << ""["" << _ << ""] : "" << v[_] << '\n'; } using ll = ; using P = pair<ll,ll>; ll MOD = 1000000007; solve() { a, b, c; cin >> a >> b >> c; if (a == b) { outl(c); } else if (a == c) { outl(b); } else { outl(a); } } main() { cin.tie(0); ios::sync_with_stdio(false); srand(()time(NULL)); fix(12); solve(); }",92
3,; main(){ x; cin >> x; if (isupper(x)){ cout << 'A' << endl; }else{ cout << 'a' << endl; } },309
4,N; ll solve() { return 180 * (N - 2); } main() { cin >> N; cout << solve() << endl; return 0; },448
...,...,...
237495,"static ll INF = 1e15; #define rep(i, n) for ( i = 0; i < ()(n); i++) ll MOD = 1e9+7; main(){ N, M; cin >> N >> M; A(N), B(N), C(M), D(M), ans(N); for( i = 0; i < N; i++) cin >> A[i] >> B[i]; for( i = 0; i < M; i++) cin >> C[i] >> D[i]; for( i = 0; i < N; i++){ dis = 1e9, cp = 1; a = A[i], b = B[i]; for( j = 0; j < M; j++){ c = C[j], d = D[j]; temp = abs(c-a) + abs(d-b); if(temp < dis) { dis = temp; cp = j+1; } } ans[i] = cp; } for(auto x : ans) cout << x << endl; }",104
237496,"# include <algorithm> # include <cassert> # include <cctype> # include <cstdio> # include <cstdlib> # include <cstring> # include <cmath> # include <iostream> # include <map> # include <queue> # include <set> # include <sstream> # include <stack> # include <string> # include # include <iomanip> # define rep(i, n) for ( i = 0; i < ()(n); i++) # define irep(i, n) for ( i = (n) - 1; i >= 0; i--) # define FOR(i, m, n) for ( i = (m); i < ()(n); i++) # define rep(i, n) for ( i = 0; i < ()(n); i++) main(){ n,len,priv,cur; v; ::iterator low; len = 0; priv = -1; cin >> n; rep(i,n){ cin >> cur; if(cur>priv){ v.push_back(cur); len++; }else{ low = lower_bound(v.begin(), v.end(), cur); *low = cur; } priv = *(v.end()-1); } cout << len << endl; }",272
237497,"#define rep(i, n, init) for ( i = init, len = (n); i < len; ++i) #define ALL(x) (x).begin(), (x).end() #define _GLIBCXX_DEBUG #define #define #define len(x) (()(x).size()) #define ALL(x) (x).begin(), (x).end() template <class Key, class Value> pair<Key, Value> findMaxValuePair(unordered_map<Key, Value> &x) { return *max_element( x.begin(), x.end(), []( pair<Key, Value> &p1, pair<Key, Value> &p2) { return p1.second < p2.second; }); } string padStart( target, len, padString) { ostringstream ss; ss << setfill(padString) << right << setw(len) << target << flush; return ss.s

### 1차 토큰화(코드 길이 제한)

In [65]:
# from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('neulab/codebert-cpp')
tokenizer.truncation_side = 'left'

# train_df와 val_df 데이터프레임 복사
train_df_copy = train_df.copy()
val_df_copy = val_df.copy()

# 토크나이저를 사용하여 코드를 토큰화
train_df_copy['code_tokenized'] = train_df_copy['code'].apply(lambda x: tokenizer.tokenize(x))
val_df_copy['code_tokenized'] = val_df_copy['code'].apply(lambda x: tokenizer.tokenize(x))

# 코드 길이를 최대 512로 제한
train_df_copy['code_tokenized'] = train_df_copy['code_tokenized'].apply(lambda x: x[:512])
val_df_copy['code_tokenized'] = val_df_copy['code_tokenized'].apply(lambda x: x[:512])

# 새로운 데이터프레임을 사용하여 pair 생성 함수 호출
bm25_train_df = get_pair(train_df_copy, tokenizer)
bm25_val_df = get_pair(val_df_copy, tokenizer)

# 결과 출력
bm25_train_df


Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors
100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [14:07<00:00,  1.70s/it]


,code1,code2,similar
0,"#define rep(i, n) for(ll i = 0; i < (n); i++) #define revrep(i, n) for(ll i = (n)-1; i >= 0; i--) #define pb push_back #define f first #define s second ll INFL = 1LL << 60; INF = 1 << 30; ll MOD = 1000000007; dy = {0, 0, 1, -1, 1, 1, -1, -1, 0}; dx = {1, -1, 0, 0, 1, -1, 1, -1, 0}; BinarySay(ll x, ll y = 60){rep(i, y) cout << (x>>(y-1-i) & 1); cout << endl;} ll pow_long(ll x, ll k){ ll res = 1; while(k > 0){ if(k % 2) res *= x; x *= x; k /= 2; } return res; } ll pow_mod(ll x, ll k){ ll res = 1; while(k > 0){ if(k % 2){ res *= x; res %= MOD; } x *= x; x %= MOD; k /= 2; } return res; } ll inverse(ll x){return pow_mod(x, MOD - 2);}; ll gcd(ll a, ll b){ if(b == 0) return a; return gcd(b, a % b); } ll lcm(ll x, ll y){return x / gcd(x, y) * y;}; ll kai_mod(ll x){ if(x == 0) return 1; return x * kai_mod(x-1) % MOD; } MAXcomb = 200010; ll fac[MAXcomb], finv[MAXcomb], inv[MAXcomb]; COMinit(){ fac[0] = fac[1] = 1; finv[0] = finv[1] = 1; inv[1] = 1; for( i = 2; i < MAXcomb; i++){ fac[i] = fac[i-1] * i % MOD; inv[i] = MOD - inv[MOD%i] * (MOD/i) % MOD; finv[i] = finv[i-1] * inv[i] % MOD; } } ll comb( n, k){ if(n < k) return 0; if(n < 0 || k < 0) return 0; return fac[n] * finv[k] % MOD * finv[n-k] % MOD; } ll N, K; A; solve(){ SUM(N+1); SUM[0] = 0; rep(i, N) SUM[i+1] = SUM[i] + A[i]; canMake; for(ll i = 1; i <= N-1; i++){ for(ll j = i; j <= N; j++){ canMake.pb(SUM[j] - SUM[i-1]); } } ll ans = 0; for(ll i = 60; i >= 0; i--){ sort(canMake.begin(), canMake.end()); ll x = lower_bound(canMake.begin(), canMake.end(), 1LL<<i) - canMake.begin(); ll cnt = canMake.size() - x; if(cnt >= K){ ans += (1LL<<i); next; for(ll j = x; j < canMake.size(); j++){ next.pb(canMake[j] - (1LL<<i)); } canMake = next; }else{ for(ll j = x; j < canMake.size(); j++){ canMake[j] -= (1LL<<i); } } } cout << ans << endl; } main(){ cin >> N >> K; A.resize(N); rep(i, N) cin >> A[i]; solve(); }","main(){ N, K; scanf(""%d"", &N); scanf(""%d"", &K); std:: A(N); std:: Asum(N+2); Asum[0]=0; for ( i=0; i<N; i++){ scanf(""%d"", &A[i]); Asum[i+1]=Asum[i]+( )A[i]; } std:: cand; for ( i=0; i<N; i++){ for ( j=i+1; j<N+1; j++){ cand.push_back(Asum[j]-Asum[i]); } } for ( i=45; i>=0; i--){ std:: cand_orig(cand.size()); std::copy(cand.begin(), cand.end(), cand_orig.begin()); cand.clear(); unit = ( )1<<i; for ( j=0; j<cand_orig.size(); j++){ if ((unit & cand_orig[j]) !=0) cand.push_back(cand_orig[j]); } if (cand.size()<K){ std::swap(cand, cand_orig); } } ans=(( )1<<45)-1; for ( j=0; j<K; j++){ ans = (ans & cand[j]); } std::cout << ans << std::endl; return 0; }",1
1,#define a[15]; main() { for ( i=1;i<=12;i++) cin>>a[i]; for ( i=1;i<=12;i++) { cout<<a[i]; if (i==4) cout<<' '; } cout<<endl; return 0; },"main(){ a1,a2,a3,a4,b1,b2,b3,b4,b5,b6,b7,b8; cin >> a1 >> a2 >> a3 >> a4 >> b1 >> b2 >> b3 >> b4 >> b5 >> b6 >> b7 >> b8; cout << a1 << a2 << a3 << a4 << ' ' << b1 << b2 << b3 << b4 << b5 << b6 << b7 << b8 << endl; }",1
2,"#define INF 1e+9 main(){ dp[1001][1001] = {}; string s1,s2; cin >> s1 >> s2; for( i = 0;i < s1.length();i++){ for( j = 0;j < s2.length();j++){ dp[i][j] = INF; } } dp[0][0] = 0; for( i = 1;i <= s1.length();i++) dp[i][0] = i; for( i = 1;i <= s2.length();i++) dp[0][i] = i; for( i = 1;i <= s1.length();i++){ for( j = 1;j <= s2.length();j++){ if(s1[i - 1] == s2[j - 1]) dp[i][j] = dp[i - 1][j - 1]; else{ dp[i][j] = min(min(dp[i - 1][j],dp[i][j - 1]),dp[i - 1][j - 1]) + 1; } } } cout << dp[s1.length()][s2.length()] << endl; return 0; }","#define rep(i, n) for (ll i = 0; i < (n); ++i) #define rrep(i, n) for (ll i = 1; i <= (n); ++i) #define irep(it, stl) for (auto it = stl.begin(); it != stl.end(); it++) #define drep(i, n) for (ll i = (n)-1; i >= 0; --i) #define mes(a) cout << (a) << endl #define dmes(a, b) cout << (a) << "" "" << (b) << endl #define re0 return 0 #define mp(p, q) make_pair(p, q) #define pb(n) push_back(n) #define all(a) a.begin(), a.end() #define rall(a) a.rbegin(), a.rend() #define Sort(a) sort(a.begin(), a.end()) #define rSort(a) sort(a.rbegin

In [4]:
# import pandas as pd
# from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('neulab/codebert-cpp')
tokenizer.truncation_side = 'left'

# test.csv 파일을 읽어들임
test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))

# 데이터프레임을 복사
test_df_copy = test_df.copy()

# 코드를 토큰화하여 새로운 열에 저장
test_df_copy['code1_tokenized'] = test_df_copy['code1'].apply(lambda x: tokenizer.tokenize(x)[:512])
test_df_copy['code2_tokenized'] = test_df_copy['code2'].apply(lambda x: tokenizer.tokenize(x)[:512])

# 토큰화된 결과를 CSV 파일에 저장
test_df_copy.to_csv(os.path.join(data_path, "test_df_2403262033.csv"), index=False)


Token indices sequence length is longer than the specified maximum sequence length for this model (516 > 512). Running this sequence through the model will result in indexing errors


In [66]:
bm25_train_df.to_csv(os.path.join(data_path, "bm25_train_df_2403262033.csv"), index=False)
bm25_val_df.to_csv(os.path.join(data_path, "bm25_val_df_2403262033.csv"), index=False)

In [68]:
base_path

'./open'

In [17]:
# !pip install rank_bm25
# !pip install datasets
# ! pip install -U accelerate
# ! pip install -U transformers

  Using cached accelerate-0.28.0-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.2 requires pytorch-lightning<1.10.0,>=1.9.0, which is not installed.
autogluon-multimodal 0.8.2 requires tensorboard<3,>=2.9, which is not installed.
autogluon-multimodal 0.8.2 requires accelerate<0.17,>=0.9, but you have accelerate 0.28.0 which is incompatible.
autogluon-multimodal 0.8.2 requires jsonschema<4.18,>=4.14, but you have jsonschema 4.21.1 which is incompatible.
autogluon-multimodal 0.8.2 requires pandas<1.6,>=1.4.1, but you have pandas 2.0.3 which is incompatible.
autogluon-multimodal 0.8.2 requires scikit-learn<1.3,>=1.0, but you have scikit-learn 1.3.2 which is incompatible.
autogluon-multimodal 0.8.2 requires torch<1.14,>=1.9, but you have torch 2.0.0+cu117 which is incompatible.
autogluon-multimodal 0.8.2 requires torchvision<0.15.0, but you have torchvision 0.15.1+cu117 w

     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     -------- ---------------------------- 30.7/134.8 kB 325.1 kB/s eta 0:00:01
     ---------------- -------------------- 61.4/134.8 kB 465.5 kB/s eta 0:00:01
     ------------------------------------ 134.8/134.8 kB 883.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.8 MB 9.8 MB/s eta 0:00:01
   ---- ----------------------------------- 1.0/8.8 MB 10.0 MB/s eta 0:00:01
   ------ --------------------------------- 1.4/8.8 MB 9.8 MB/s eta 0:00:01
   ------- -------------------------------- 1.6/8.8 MB 9.5 MB/s eta 0:00:01
   --------- ------------------------------ 2.1/8.8 MB 9.5 MB/s eta 0:00:01
   ------------ --------------------------- 2.7/8.8 MB 9.5 MB/s eta 0:00:01
   -----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.2 requires pytorch-lightning<1.10.0,>=1.9.0, which is not installed.
autogluon-multimodal 0.8.2 requires tensorboard<3,>=2.9, which is not installed.
autogluon-multimodal 0.8.2 requires accelerate<0.17,>=0.9, but you have accelerate 0.28.0 which is incompatible.
autogluon-multimodal 0.8.2 requires jsonschema<4.18,>=4.14, but you have jsonschema 4.21.1 which is incompatible.
autogluon-multimodal 0.8.2 requires pandas<1.6,>=1.4.1, but you have pandas 2.0.3 which is incompatible.
autogluon-multimodal 0.8.2 requires scikit-learn<1.3,>=1.0, but you have scikit-learn 1.3.2 which is incompatible.
autogluon-multimodal 0.8.2 requires torch<1.14,>=1.9, but you have torch 2.0.0+cu117 which is incompatible.
autogluon-multimodal 0.8.2 requires torchvision<0.15.0, but you have torchvision 0.15.1+cu117 w

In [18]:
# !pip install tensorflow

  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow-2.13.0-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
  Using cached tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached protobuf-4.25.3-cp38-cp38-win_amd64.whl.metadata (541 bytes)
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached wrapt-1.16.0-cp38-cp38-win_amd64.whl.metadata (6.8 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached keras-2.13.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached tensorboard_data_server-0.7.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.2 requires pytorch-lightning<1.10.0,>=1.9.0, which is not installed.
dash 2.14.1 requires ansi2html, which is not installed.
dash 2.14.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.14.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.14.1 requires dash-table==5.0.0, which is not installed.
dash 2.14.1 requires retrying, which is not installed.
mljar-supervised 1.1.1 requires xgboost>=2.0.0, which is not installed.
ray 2.3.1 requires click>=7.0, which is not installed.
typer 0.9.0 requires click<9.0.0,>=7.1.1, which is not installed.
autogluon-multimodal 0.8.2 requires accelerate<0.17,>=0.9, but you have accelerate 0.28.0 which is incompatible.
autogluon-multimodal 0.8.2 requires jsonschema<4.18,>=4.14, but you have jsonschema 4.21.1 which is in

### 학습 준비

In [1]:

import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import torch
import transformers

from glob import glob
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, AutoModel, AutoModelForSequenceClassification,DataCollatorForTokenClassification,EarlyStoppingCallback
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric, Dataset

from tqdm import tqdm
from tqdm import trange
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
data_path = 'D:/Users/coding/open/dataframe/'


train_df = pd.read_csv(os.path.join(data_path, "bm25_train_df_2403262033.csv"))
val_df = pd.read_csv(os.path.join(data_path, "bm25_val_df_2403262033.csv"))
train_df = train_df.rename(columns={'similar': 'label'})
val_df = val_df.rename(columns={'similar': 'label'})

In [7]:
train_df

,code1,code2,label
0,"#define rep(i, n) for(ll i = 0; i < (n); i++) ...","main(){ N, K; scanf(""%d"", &N); scanf(""%d"", &K)...",1
1,#define a[15]; main() { for ( i=1;i<=12;i++) c...,"main(){ a1,a2,a3,a4,b1,b2,b3,b4,b5,b6,b7,b8; c...",1
2,#define INF 1e+9 main(){ dp[1001][1001] = {}; ...,"#define rep(i, n) for (ll i = 0; i < (n); ++i)...",0
3,#define INF 999999999999999997 #define MP make...,"main(){ n,m; cin >> n >> m; ac,tle; ac=n-m; tl...",1
4,using ll = ; INFint = 2e9+1; ll INFll = 2e18+1...,"#define repl(i, l, r) for (ll i = (l); i < (r)...",0
...,...,...,...
5426495,using ll= ; #define _CRT_SECURE_NO_WARNINGS #d...,"#define SORT(c) sort((c).begin(),(c).end()) #d...",0
5426496,#define mp make_pair #define pb push_back #def...,#define maxn 200010 maxm=1<<26; #define INF 0x...,0
5426497,"main(){ n,leg[2]={0,0},count=0; state=false; f...","main() { string a[100]; b; while (cin >> b, b)...",1
5426498,"> G[100000]; ll K; d(100000, LLONG_MAX); prior...","#define REP(i,n) for( i=0, i##_len=(n); i<i##_...",0


In [24]:
train_df = train_df.sample(n = 20000000, replace = True).reset_index(drop=True)
val_df = val_df.sample(n= 1000, replace = True).reset_index(drop = True)
dataset_train = Dataset.from_pandas(train_df)
dataset_val = Dataset.from_pandas(val_df)

In [27]:
dataset_train

Dataset({
    features: ['code1', 'code2', 'label'],
    num_rows: 20000000
})

### 2차 토큰화

In [ ]:
import random

#model_name = 'neulab/codebert-cpp'
model_name = 'microsoft/codereviewer'
wd = 0.01
batch_size = 16
lr = 2e-5
epochs = 1
task = 'binary_classification'
label_list = ['0', '1']
num_labels = 2

def seed_everything(seed:42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of T5ForSequenceClassification were not initialized from the model checkpoin

In [ ]:
def tokenize_function(examples):
    tokenizer.truncation_side = 'left'
    return tokenizer(examples["code1"], examples["code2"],padding="max_length", max_length = 512, truncation=True)

In [ ]:
#tokenized_train_datasets= dataset_train.map(tokenize_function, batched=True)
tokenized_val_datasets = dataset_val.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### 학습(coderiviewer)

In [ ]:
args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/Colab Notebooks/open/codereviewer',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate= lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    seed = 42,
    weight_decay=wd,
    load_best_model_at_end=True,
    logging_dir = '/content/drive/MyDrive/Colab Notebooks/open/codereviewer'
)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = []
    for i in pred.predictions:
      preds.append(np.argmax(i, axis=1).flatten())
    print(preds)
    acc = accuracy_score(labels, preds[0])
    return {'accuracy': acc}

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_val_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
trainer.train()

NameError: name 'tokenized_train_datasets' is not defined

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/open/codereviewer'
data_path = '/content/drive/MyDrive/Colab Notebooks/open/dataframe'

In [ ]:
model = torch.load(os.path.join(base_path, 'model0311.pt'))

tokenizer = AutoTokenizer.from_pretrained(model_name)

test_df = pd.read_csv(os.path.join(data_path,'test_df_2403262033.csv'))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test_df = test_df.drop(columns = ['pair_id', 'code1_tokenized', 'code2_tokenized'], axis = 1)

In [ ]:
for i in range(len(test_df)):
  if i % 10000 == 0:
    print(i)
  test_df.iloc[i]['code1'] = data_clean(test_df.iloc[i]['code1'])
  test_df.iloc[i]['code2'] = data_clean(test_df.iloc[i]['code2'])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000


In [ ]:
args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/Colab Notebooks/open',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate= lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    seed = 42,
    weight_decay=wd,
    load_best_model_at_end=True,
    logging_dir = '/content/drive/MyDrive/Colab Notebooks/open'
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = []
    for i in pred.predictions:
      preds.append(np.argmax(i, axis=1).flatten())
    print(preds)
    acc = accuracy_score(labels, preds[0])
    return {'accuracy': acc}

trainer = Trainer(
    model,
    args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


### 예측

In [ ]:
predictions = []
for i in trange(len(test_df) // 5000):
  pred = []
  dataset_test = Dataset.from_pandas(test_df[i*5000:(i+1)*5000])
  tokenized_test_datasets = dataset_test.map(tokenize_function, batched=True)
  predictions_test = trainer.predict(tokenized_test_datasets)

  for j in predictions_test.predictions:
    pred.append(np.argmax(j, axis=1).flatten())

  predictions.extend(pred[0])

len(predictions)

  0%|          | 0/119 [00:00<?, ?it/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

  1%|          | 1/119 [02:05<4:06:26, 125.31s/it]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

  2%|▏         | 2/119 [03:53<3:44:28, 115.11s/it]

...

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

 99%|█████████▉| 118/119 [3:31:52<01:47, 107.62s/it]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

100%|██████████| 119/119 [3:33:39<00:00, 107.73s/it]


595000

In [ ]:
#public 0.76592 (codebert - test전처리 전)
#public 0.81441 (codebert - test전처리 후)
#public xx (codereviewer - test전처리 후)

In [ ]:
sub = pd.read_csv(os.path.join('/content/drive/MyDrive/Colab Notebooks/open', 'sample_submission.csv'))
sub['similar'] = predictions

sub.to_csv(os.path.join(base_path, 'submission_0313.csv'), index = False)

In [ ]:
tokenized_val_datasets = dataset_val.map(tokenize_function, batched=True)

Map:   0%|          | 0/55503 [00:00<?, ? examples/s]

In [ ]:
predictions_val = trainer.predict(tokenized_val_datasets)

[array([0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0

In [ ]:
preds = []
for i in predictions_val.predictions:
  preds.append(np.argmax(i, axis=1).flatten())

predict = preds[0]

In [ ]:
score = 0
for i in range(len(predict)):
  if val_df.iloc[i]['label'] == predict[i]:
    score += 1

score / len(predict)

0.981